In [ ]:
import os
import scanpy as sc
import pandas as pd
import os
import conga
import glob
import re
# %load_ext rpy2.ipython

import sys
sys.path.append('/home/jovyan/conga/my_scripts/python/')

In [ ]:
experiments_dir = "/home/jovyan/conga/data/20118016/"

# Change the working directory
os.chdir(experiments_dir)

tcr_meta = [experiments_dir + 'raw/tcr_meta_nil.csv',
            experiments_dir + 'raw/tcr_meta_til.csv',
            experiments_dir + 'raw/tcr_meta_pbl.csv',
            experiments_dir + 'raw/tcr_meta_sample1.csv',
            experiments_dir + 'raw/tcr_meta_sample2.csv',
            experiments_dir + 'raw/tcr_meta_sample3.csv']

h5_files = [experiments_dir + 'raw/nil_filtered_feature_bc_matrix.h5',
            experiments_dir + 'raw/til_filtered_feature_bc_matrix.h5',
            experiments_dir + 'raw/pbl_filtered_feature_bc_matrix.h5',
            experiments_dir + 'raw/sample1_filtered_feature_bc_matrix.h5',
            experiments_dir + 'raw/sample2_filtered_feature_bc_matrix.h5',
            experiments_dir + 'raw/sample3_filtered_feature_bc_matrix.h5']

results_dirs = ['results_nil',
                'results_til',
                'results_pbl',
                'results_sample1',
                'results_sample2',
                'results_sample3']

In [ ]:
from make_conga_clone import process_tcr

clones_files = []

for tcr, results in zip(tcr_meta, results_dirs):
    clones_file = process_tcr(tcr, results_dir=results, perform_kpca="auto")
    clones_files.append(clones_file)

In [ ]:
# %%R -i clones_files -i h5_files

# source('/home/jovyan/conga/scripts/R/conga_seurat.R')

# # Initialize a list to store the merged counts matrices
# merged_counts_matrix_files <- list()

# # Ensure that the lengths of both lists are the same
# if(length(clones_files) != length(h5_files)) {
#     stop("The number of clone files and H5 files must be the same.")
# }

# for (i in 1:length(clones_files)) {
#     # Get the current clone file and corresponding H5 file
#     current_clone_file <- clones_files[i]
#     current_h5_file <- h5_files[i]

#     # Process the current pair of files
#     temp_matrix <- process_seurat_data(current_clone_file, current_h5_file)
    
#     # Store the result in the list
#     merged_counts_matrix_files[[i]] <- temp_matrix
# }

# # # #merged_counts_matrix_files now contains a separate matrix for each pair of clone file and H5 file


In [ ]:
# adata_list = list()

# for file in h5_files:

#     split_fn = file.split('_')

#     sample_name = f'{split_fn[1]}_{split_fn[2]}'

#     clones_file = f'{sample_name}_clones.tsv'

#     xdata = conga.preprocess.read_dataset(

#         file,

#         '10x_h5',

#         clones_file,

#         allow_missing_kpca_file=True)

 

#     xdata.obs['sample_name'] = sample_name

#     xdata.obs['condition'] = 'HBV' if sample_name in HBV_pos else 'healthy'

#     xdata.obs['condition_batch'] = 1 if sample_name in HBV_pos else 0

#     adata_list.append(xdata)

In [14]:
import traceback

# Initialize a list to store the merged counts matrices
adata_list = []

# Ensure that the lengths of both lists are the same
if len(clones_files) != len(h5_files):
    raise ValueError("The number of clone files and H5 files must be the same.")
s
for i in range(len(clones_files)):
    current_clone_file = clones_files[i]
    current_h5_file = h5_files[i]

    print(f"-----------------------------------------")
    print(f"Starting run #{i}")
    print(f"Current clone file: {current_clone_file}")
    print(f"Current H5 file: {current_h5_file}")

    if not os.path.exists(current_clone_file) or not os.path.exists(current_h5_file):
        print(f"Error: File not found - Clone file: {current_clone_file}, H5 file: {current_h5_file}")
        continue

    try:
        xdata = conga.preprocess.read_dataset(
            current_h5_file,
            '10x_h5',
            current_h5_file,
            allow_missing_kpca_file=True
        )
        
        # Check and make variable names unique
        num_var_names_before = len(xdata.var_names)
        num_unique_var_names_before = len(set(xdata.var_names))
        xdata.var_names_make_unique()
        num_var_names_after = len(xdata.var_names)
        num_unique_var_names_after = len(set(xdata.var_names))

        print(f"Variable names before: {num_var_names_before}, Unique: {num_unique_var_names_before}")
        print(f"Variable names after: {num_var_names_after}, Unique: {num_unique_var_names_after}")
            
        adata_list.append(xdata)
        
    except AssertionError as error:
        print(f"An assertion error occurred.")
        print(f"Failed at clone file: {current_clone_file}")
        print(f"Failed at H5 file: {current_h5_file}")
        traceback.print_exc() 

-----------------------------------------
Starting run #0
Current clone file: /home/jovyan/conga/data/20118016/raw/results_nil/merged_clones.tsv
Current H5 file: /home/jovyan/conga/data/20118016/raw/nil_filtered_feature_bc_matrix.h5
reading: /home/jovyan/conga/data/20118016/raw/nil_filtered_feature_bc_matrix.h5 of type 10x_h5


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError


total barcodes: 3286 (3286, 36601)
An assertion error occurred.
Failed at clone file: /home/jovyan/conga/data/20118016/raw/results_nil/merged_clones.tsv
Failed at H5 file: /home/jovyan/conga/data/20118016/raw/nil_filtered_feature_bc_matrix.h5
-----------------------------------------
Starting run #1
Current clone file: /home/jovyan/conga/data/20118016/raw/results_til/merged_clones.tsv
Current H5 file: /home/jovyan/conga/data/20118016/raw/til_filtered_feature_bc_matrix.h5
reading: /home/jovyan/conga/data/20118016/raw/til_filtered_feature_bc_matrix.h5 of type 10x_h5


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError


total barcodes: 3255 (3255, 36601)
An assertion error occurred.
Failed at clone file: /home/jovyan/conga/data/20118016/raw/results_til/merged_clones.tsv
Failed at H5 file: /home/jovyan/conga/data/20118016/raw/til_filtered_feature_bc_matrix.h5
-----------------------------------------
Starting run #2
Current clone file: /home/jovyan/conga/data/20118016/raw/results_pbl/merged_clones.tsv
Current H5 file: /home/jovyan/conga/data/20118016/raw/pbl_filtered_feature_bc_matrix.h5
reading: /home/jovyan/conga/data/20118016/raw/pbl_filtered_feature_bc_matrix.h5 of type 10x_h5


...

Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError


total barcodes: 4686 (4686, 36601)
An assertion error occurred.
Failed at clone file: /home/jovyan/conga/data/20118016/raw/results_sample1/merged_clones.tsv
Failed at H5 file: /home/jovyan/conga/data/20118016/raw/sample1_filtered_feature_bc_matrix.h5
-----------------------------------------
Starting run #4
Current clone file: /home/jovyan/conga/data/20118016/raw/results_sample2/merged_clones.tsv
Current H5 file: /home/jovyan/conga/data/20118016/raw/sample2_filtered_feature_bc_matrix.h5
reading: /home/jovyan/conga/data/20118016/raw/sample2_filtered_feature_bc_matrix.h5 of type 10x_h5


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError


total barcodes: 3978 (3978, 36601)
An assertion error occurred.
Failed at clone file: /home/jovyan/conga/data/20118016/raw/results_sample2/merged_clones.tsv
Failed at H5 file: /home/jovyan/conga/data/20118016/raw/sample2_filtered_feature_bc_matrix.h5
-----------------------------------------
Starting run #5
Current clone file: /home/jovyan/conga/data/20118016/raw/results_sample3/merged_clones.tsv
Current H5 file: /home/jovyan/conga/data/20118016/raw/sample3_filtered_feature_bc_matrix.h5
reading: /home/jovyan/conga/data/20118016/raw/sample3_filtered_feature_bc_matrix.h5 of type 10x_h5
total barcodes: 4595 (4595, 36601)
An assertion error occurred.
Failed at clone file: /home/jovyan/conga/data/20118016/raw/results_sample3/merged_clones.tsv
Failed at H5 file: /home/jovyan/conga/data/20118016/raw/sample3_filtered_feature_bc_matrix.h5


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1899: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
Traceback (most recent call last):
  File "/tmp/ipykernel_78515/2971089593.py", line 24, in <module>
    xdata = conga.preprocess.read_dataset(
  File "/home/jovyan/conga/conga/preprocess.py", line 309, in read_dataset
    assert exists(bcmap_file)
AssertionError
